<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Surprise Singular Value Decomposition (SVD)

This notebook reviews the SVD algorithm in the [Surprise library](http://surpriselib.com/).  SVD is a widely used collaborative filtering method,  and part of [the winning solution](https://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf) in the Netflix Prize competition. 

## 1 Singular Value Decomposition

SVD is a *matrix factorziation* technique, which we briefly review in the [ALS deep dive notebook](als_deep_dive.ipynb).  SVD is similar to ALS (Alternating Least Squares), with the following differences:

[comment]: <> (Should we add references below?)

- SVD additionally models the user and item biases.
- The optimization technique in SVD is stochastic gradient descent, as opposed to Alternating Least Squares.

### 1.1 The SVD model

In ALS, the ratings are modeled as follows:

$$\hat r_{u,i} = q_{i}^{T}p_{u}$$

SVD introduces two new scalar variables: the user biases $b_u$ and the item biases $b_i$. The user biases are supposed to capture the tendency of some users to rate items higher (or lower) than the average. The same goes for items: some items are usually rated higher than some others. The model is SVD is then as follows:

$$\hat r_{u,i} = \mu + b_u + b_i + q_{i}^{T}p_{u}$$

Where $\mu$ is the global average of all the ratings in the dataset. The regularised optimization problem naturally becomes:

$$ \sum(r_{u,i} - (\mu + b_u + b_i + q_{i}^{T}p_{u}))^2 + \lambda(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2)$$

where $\lambda$ is a the regularization parameter.


### 1.2 Stochastic Gradient Descent

Stochastic Gradient Descent (SGD) is a very common algorithm for optimization where the parameters (here the biases and the factor vectors) are iteratively incremented with the negative gradients w.r.t the optimization function. The algorithm essentially performs the following steps for a given number of iterations:
$$b_u \leftarrow b_u + \gamma (e_{ui} - \lambda b_u)$$
$$b_i \leftarrow b_i + \gamma (e_{ui} - \lambda b_i)$$
$$p_u \leftarrow p_u + \gamma (e_{ui} \cdot q_i - \lambda p_u)$$
$$q_i \leftarrow q_i + \gamma (e_{ui} \cdot p_u - \lambda q_i)$$
where $\gamma$ is the learning rate and $e_{ui} =  r_{ui} - \hat r_{u,i} = r_{u,i} - (\mu + b_u + b_i + q_{i}^{T}p_{u})$ is the error made by the model for the pair $(u, i)$.

## 2 Surprise Implementation of SVD

SVD is implemented in the [Surprise](https://surprise.readthedocs.io/en/stable/) library as a recommender module. 
* Detailed documentations of the SVD module in Surprise can be found [here](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD).
* Source codes of the SVD implementation is available on the Surprise Github repository, which can be found [here](https://github.com/NicolasHug/Surprise/blob/master/surprise/prediction_algorithms/matrix_factorization.pyx).

## 3 Surprise SVD Movie Recommender

We will use the Movielens dataset, which is composed of integer ratings from 1 to 5. 

Surprise supports dataframes as long as they have three colums reprensenting the user ids, item ids, and the ratings (in this order).

In [5]:
import sys
sys.path.append("../../")
import time
import os
import surprise
import papermill as pm
import pandas as pd
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Surprise version: 1.0.6


In [6]:
# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### 3.1 Load and split data

In [7]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

,userID,itemID,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


We then split our data into trainset and testset with the `python_random_split` function.

In [8]:
train, test = python_random_split(data, 0.75)

### 3.2 Training a SVD model

Note that Surprise has a lot of built-in support for [cross-validation](https://surprise.readthedocs.io/en/stable/getting_started.html#use-cross-validation-iterators) or also [grid search](https://surprise.readthedocs.io/en/stable/getting_started.html#tune-algorithm-parameters-with-gridsearchcv) inspired scikit-learn, but we will here use the tools provided in `reco_utils` instead.

Surprise needs to build an internal model of the data. Here, we use the `load_from_df` method to build a `Dataset` object, and then indicate that we want to train on all the samples of this dataset by using the `build_full_trainset` method.

In [9]:
# 'reader' is being used to get rating scale (for MovieLens, the scale is [1, 5]).
# 'rating_scale' parameter can be used instead for the later version of surprise lib:
# https://github.com/NicolasHug/Surprise/blob/master/surprise/dataset.py
train_set = surprise.Dataset.load_from_df(train, reader=surprise.Reader('ml-100k')).build_full_trainset()

The [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD) has a lot of parameters. The most important ones are:
- `n_factors`, which controls the dimension of the latent space (i.e. the size of the vectors $p_u$ and $q_i$). Usually, the quality of the training set predictions grows with as `n_factors` gets higher.
- `n_epochs`, which defines the number of iteration of the SGD procedure.

Note that both parameter also affect the training time.


We will here set `n_factors` to `200` and `n_epochs` to `30`. To train the model, we simply need to call the `fit()` method.

In [10]:
svd = surprise.SVD(random_state=0, n_factors=200, n_epochs=30, verbose=True)

start_time = time.time()

svd.fit(train_set)

train_time = time.time() - start_time
print("Took {} seconds for training.".format(train_time))

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Took 7.49165940284729 seconds for training.


### 3.3 Prediction

Now that our model is fitted, we can call `predict` to get some predictions. `predict` returns an internal object `Prediction` which can be easily converted back to a dataframe:

In [11]:
predictions = [svd.predict(row.userID, row.itemID, row.rating)
               for (_, row) in test.iterrows()]

test_time = time.time() - start_time

predictions = pd.DataFrame(predictions)
predictions = predictions.rename(index=str, columns={'uid': 'userID', 'iid': 'itemID',
                                                     'est': 'prediction'})
predictions = predictions.drop(['details', 'r_ui'], axis='columns')
for col in ['userID', 'itemID']:
    predictions[col] = predictions[col].astype(int)
predictions.head()

,userID,itemID,prediction
0,877,381,3.698217
1,815,602,3.590957
2,94,431,3.841149
3,416,875,2.642248
4,500,182,4.384139


To compute ranking metrics, we need predictions on all user, item pairs. We remove though the items already watched by the user, since we choose not to recommend them again. 

In [12]:
start_time = time.time()

preds_lst = []
for user in train.userID.unique():
    for item in train.itemID.unique():
        preds_lst.append([user, item, svd.predict(user, item).est])

all_predictions = pd.DataFrame(data=preds_lst, columns=["userID", "itemID", "prediction"])

merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

test_time = time.time() - start_time
print("Took {} seconds for prediction.".format(test_time))

Took 10.710219860076904 seconds for prediction.


In [13]:
all_predictions.head()

,userID,itemID,prediction
75000,811,755,4.090273
75001,811,287,4.557071
75002,811,181,4.571596
75003,811,96,4.458827
75004,811,83,4.559237


### 3.4 Evaluation 

The SVD algorithm was specifically designed to predict ratings as close as possible to their actual values. In particular, it is designed to have a very low RMSE (Root Mean Squared Error), computed as:

$$\sqrt{\frac{1}{N} \sum(\hat{r_{ui}} - r_{ui})^2}$$

As we can see, the RMSE and MAE (Mean Absolute Error) are pretty low (i.e. good), indicating that on average the error in the predicted ratings is less than 1. The RMSE is of course a bit higher, because high errors are penalized much more.

For comparison with other models, we also display Top-k and ranking metrics (MAP, NDCG, etc.). Note however that the SVD algorithm was designed for achieving high accuracy, not for top-rank predictions.

In [16]:
eval_rmse = rmse(test, predictions)
eval_mae = mae(test, predictions)
eval_rsquared = rsquared(test, predictions)
eval_exp_var = exp_var(test, predictions)

k = 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)


print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

print('----')

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

RMSE:		0.948771
MAE:		0.747003
rsquared:	0.288045
exp var:	0.288157
----
MAP:	0.015624
NDCG:	0.110465
Precision@K:	0.100425
Recall@K:	0.035267


In [18]:
# Record results with papermill for tests
pm.record("rmse", eval_rmse)
pm.record("mae", eval_mae)
pm.record("rsquared", eval_rsquared)
pm.record("exp_var", eval_exp_var)
pm.record("map", eval_map)
pm.record("ndcg", eval_ndcg)
pm.record("precision", eval_precision)
pm.record("recall", eval_recall)
pm.record("train_time", train_time)
pm.record("test_time", test_time)

## Additional Reading

[Koren, Bell and Volinksy's paper in 2009](https://ieeexplore.ieee.org/document/5197422) provides a good overview of matrix factorization methods in recommendations.  Nicolas Hug, developer of the Surprise library and the author of the first versions of this notebook, has a nice 4-part [blog post](http://nicolas-hug.com/blog/matrix_facto_1) about matrix factorization and specifically SVD.